In [1]:
pip install openai

You should consider upgrading via the 'c:\Users\brinp\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import openai
import webbrowser

In [3]:
def questions(question):
    # https://beta.openai.com/account/api-keys
    openai.api_key = 
    model_engine = 'text-davinci-003'
    completions = openai.Completion.create(engine=model_engine, prompt=question, max_tokens=1024, n=1,stop=None,temperature=0)

    # https://beta.openai.com/docs/api-reference/completions/create
    message = completions.choices[0].text
    print(message)

In [4]:
def image_gen(im, amount):
  i = 0
  while i<amount:
      
      response = openai.Image.create(
        prompt=im,
        n=1,
        size="1024x1024"
      )
      image_url = response['data'][0]['url']
      image_url

      webbrowser.open(image_url)
      i+=1

In [6]:
questions(input('Ask a question: '))



#include <stdio.h>

int main(void)
{
    printf("Hi!\n");
    return 0;
}


In [8]:
image_gen(input('Generate image: '),5)

In [18]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\brinp\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
pip install git+https://github.com/openai/point-e.git

  Cloning https://github.com/openai/point-e.git to c:\users\brinp\appdata\local\temp\pip-req-build-lmkc5edx
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/openai/point-e.git 'C:\Users\brinp\AppData\Local\Temp\pip-req-build-lmkc5edx'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/openai/point-e.git 'C:\Users\brinp\AppData\Local\Temp\pip-req-build-lmkc5edx'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
You should consider upgrading via the 'c:\Users\brinp\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [20]:
%cd point-e

[WinError 2] The system cannot find the file specified: 'point-e'
c:\Users\brinp\Documents\Jupyter\GTR


In [21]:
pip install .

Note: you may need to restart the kernel to use updated packages.


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the 'c:\Users\brinp\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [23]:
#2. Package import

from PIL import Image
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud


#3. Model prep

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base1B' # Qualitat resultat = base40M < base300M < base1B (de pitjor a millor)
base_model =  model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

#4.  Sampler prep

sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)

#4. Execution of interfaces


img = Image.open('point_e/examples/example_data/cube_stack.jpg')
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
    samples = x

#5. Display of 3D point cloud

pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

ModuleNotFoundError: No module named 'point_e'

In [24]:
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 0.0],
    model_kwargs_key_filter=('texts', ''), # Do not condition the upsampler at all
)
# Set a prompt to condition on.
prompt = 'a red motorcycle'

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
    samples = x
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

ModuleNotFoundError: No module named 'point_e'